In [1]:
%cd ..

c:\Users\HP\OneDrive - University of Moratuwa\Desktop\TMS-Project\TMS-Backend


In [6]:
from sqlalchemy.orm import Session
from typing import Optional, List, Dict
from datetime import date
from pydantic import BaseModel
from src.database import models

class OptimizeResponse(BaseModel):
    job_id: int
    message: str
    fixed_routes: int
    optimized_routes: int
    total_shops: int

class VehicleRouteAssignment(BaseModel):
    vehicle_id: int
    predefined_route_id: Optional[int] = None 
    
class OptimizeRequest(BaseModel):
    day: Optional[date]
    vehicles: Optional[List[VehicleRouteAssignment]]
    selected_order_id: Optional[List[int]] = None

class ORDataModelS:
    def __init__(self, request: OptimizeRequest, db: Session):
        self.request = request
        self.db = db
        pass
    
    def _get_orders(self, ids):
        orders = self.db.query(models.Order).filter(
            models.Order.id.in_(ids)
        ).all()
        
        return orders
    
    def _get_vehicles(self, ids):
        vehicles = self.db.query(models.Vehicles).filter(
            models.Vehicles.id.in_(ids)
        ).all()
        return vehicles
    
    def _get_geo_constrain(self):
        constrain = self.db.query(models.GeoConstraint).filter().all()
        
        return constrain

In [13]:
req = {"vehicles":[{"vehicle_id":1}, {"vehicle_id":2, "predefined_route_id": 1}], "selected_order_id":[1,2,3,4]}

In [14]:
reqest = OptimizeRequest(day="2025-10-10",vehicles=[{"vehicle_id":1}, {"vehicle_id":2, "predefined_route_id": 1}],
                      selected_order_id=[1,2,3,4])

In [20]:
reqest

OptimizeRequest(day=datetime.date(2025, 10, 10), vehicles=[VehicleRouteAssignment(vehicle_id=1, predefined_route_id=None), VehicleRouteAssignment(vehicle_id=2, predefined_route_id=1)], selected_order_id=[1, 2, 3, 4])

In [23]:
reqest.vehicles[0]

VehicleRouteAssignment(vehicle_id=1, predefined_route_id=None)

In [16]:
for r in req['vehicles']:
    r

In [17]:
from src.database.database import SessionLocal
db = SessionLocal()
data_model = ORDataModelS(reqest, db=db)

In [18]:
veh = data_model._get_vehicles([1, 2])

In [19]:
for v in veh:
    print(v.constraint.max_distance)
    print(v.constraint.max_visits)

300.0
6
300.0
6


In [11]:
orders =  data_model._get_orders([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])


for o in orders:
    print(o.time_window_start)
    print(o.time_window_end)

None
None
None
None
None
None
10:00:00
15:30:00
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
08:00:00
12:00:00


In [12]:
geo =  data_model._get_geo_constrain()


for g in geo:
    print(g.start_shop_id)
    print(g.end_shop_id)
    print(g.vehicle_id)
    print("==========")

In [52]:
from sqlalchemy.orm import Session
from typing import Optional, Union, List, Dict
from datetime import datetime, time
from src.database import models
from src.core.matrix_manager import DistanceMatrixManager


class ORDataModel:
    def __init__(self, db: Session, vehicles: List[models.Vehicles], orders: List[models.Order], use_time_windows: bool=False, depot_id: int=1):
        self.db = db
        self.depot_id = depot_id
                
        self.shop_ids = [o.shop_id for o in orders]
        self.all_nodes = [self.depot_id] + self.shop_ids  # depot = 1
        
        self.distance_matrix, self.time_matrix = self._get_matrix(all_nodes=self.all_nodes)
        self.use_time = self._use_time_window(orders) or use_time_windows
        self.max_distance_per_vehicle, self.max_visits_per_vehicle = self._get_vehicle_constrains(vehicles=vehicles)
        self.penalties = self._build_penalties(orders=orders)
        self.time_windows = self._build_time_windows(orders=orders, use_tw=self.use_time)

        self.order_groups = self._fetch_order_groups(orders)
        self.geo_constrains = self._fetch_geo_constraints(self.db)
        
        self.matrix = self.time_matrix if self.use_time else self.distance_matrix
    
    
    def get_data(self):
        return {
            "matrix": self.matrix,
            "demands":[0] + [1]*len(self.shop_ids),
            "node_mapping":self.all_nodes,
            "num_vehicles":len(self.max_distance_per_vehicle),
            "depot": 0,
            "max_distance_per_vehicle": self.max_distance_per_vehicle,
            "max_visits_per_vehicle":self.max_visits_per_vehicle,
            "penalties":self.penalties,
            "order_groups": self.order_groups, 
            "geo_constrains": self.geo_constrains,
            "time_windows":self.time_windows,
            "use_time_matrix":self.use_time
        }
    
    def _get_matrix(self, all_nodes: List[int]):       
        matrix_mgr = DistanceMatrixManager(self.db)
        
        distance_matrix = matrix_mgr.get_distance_matrix_as_array(all_nodes)
        time_matrix = matrix_mgr.get_time_matrix_as_array(all_nodes)
        
        return distance_matrix, time_matrix
    
    def _use_time_window(self, orders: List[models.Order]):
        has_time_windows = any(
		o.time_window_start is not None and o.time_window_end is not None 
		for o in orders)
        
        return has_time_windows
    
    def _get_vehicle_constrains(self, vehicles: List[models.Vehicles]):
        max_distance_per_vehicle = []
        max_visits_per_vehicle = []
        
        for v in vehicles:
            if v.constraint:
                max_distance_per_vehicle.append(int(v.constraint.max_distance or 500))
                max_visits_per_vehicle.append(int(v.constraint.max_visits or 15))
            else:
                # fallback defaults
                max_distance_per_vehicle.append(500)
                max_visits_per_vehicle.append(15)
                
        return max_distance_per_vehicle, max_visits_per_vehicle


    def _build_time_windows(self, orders: List[models.Order], use_tw: bool):
        
        if not use_tw:
            return None
        
        tw = {self.depot_id: (0, 1440)}  # Depot: 00:00 to 24:00 (1440 minutes)
        
        for o in orders:
            start = self._to_minutes(getattr(o, "time_window_start", None))
            end = self._to_minutes(getattr(o, "time_window_end", None))
            
            if start is not None and end is not None:
                tw[o.shop_id] = (start, end)
            else:
                tw[o.shop_id] = (0, 1440)
                
        return tw
    
    
    def _build_penalties(self, orders: List[models.Order]):
        
        shop_priority_map = {}
        
        for order in orders:
            if hasattr(order, 'priority') and order.priority:
                priority = order.priority 
            else:
                priority = models.Priority.MEDIUM
            
            if priority == models.Priority.HIGH:
                shop_priority_map[order.shop_id] = 100000
                
            elif priority == models.Priority.LOW:
                shop_priority_map[order.shop_id] = 500 
                
            else:
                shop_priority_map[order.shop_id] = 5000 
        
        penalties = []       
        for node in self.all_nodes:
            if node == self.depot_id:
                penalties.append(0) 
            else:
                penalties.append(shop_priority_map.get(node, 5000)) 
                
        return penalties
    
    
    def _fetch_order_groups(self, orders: List[models.Order]) -> Dict[int, List[int]]:
        """Extract order groups from orders.
        
        Orders in the same group must be assigned to the same vehicle.
        
        Args:
            orders: List of order objects
        
        Returns:
            Dictionary mapping group_id -> [list of shop_ids in that group]
            Example: {1: [shop_5, shop_8, shop_12], 2: [shop_3, shop_7]}
        """
        order_groups = {}
        
        for order in orders:
            # Check if order belongs to any group (can be in multiple groups)
            if hasattr(order, 'group') and order.group:
                for group in order.group:
                    group_id = group.id
                    if group_id not in order_groups:
                        order_groups[group_id] = []
                    order_groups[group_id].append(order.shop_id)
            
        # Remove duplicates and return only groups with multiple orders
        filtered_groups = {}
        for group_id, shop_ids in order_groups.items():
            unique_shops = list(set(shop_ids))
            if len(unique_shops) > 1:  # Only enforce constraint if group has multiple orders
                filtered_groups[group_id] = unique_shops
        
        return filtered_groups
    
    def _fetch_geo_constraints(self, db: Session) -> List[Dict]:
        """Fetch geo constraints from database and structure them for the optimizer.
        
        Returns:
            List of geo constraint dicts with format:
            {
                "fromCode": shop_code,
                "toCode": shop_code,
                "restrictedVehicle": vehicle_id or None (None = applies to all vehicles)
            }
        """
        geo_constraints = db.query(models.GeoConstraint).all()
        
        structured_constraints = []
        for gc in geo_constraints:
            constraint = {
                # Prefer matching by IDs for correctness
                "fromId": gc.start_shop_id,
                "toId": gc.end_shop_id,
                # Keep codes for debugging/legacy if needed (not used in matching)
                "fromCode": str(gc.start_shop.shop_code) if gc.start_shop else None,
                "toCode": str(gc.end_shop.shop_code) if gc.end_shop else None,
                "restrictedVehicle": gc.vehicle_id  # None if applies to all vehicles
            }
            if constraint["fromId"] is not None and constraint["toId"] is not None:
                structured_constraints.append(constraint)
        
        return structured_constraints
                
    
    def _to_minutes(self, t: Optional[Union[time, str]]) -> Optional[int]:
        """Convert a datetime.time or 'HH:MM[:SS]' string into minutes since midnight.
        Returns None if value cannot be parsed.
        """
        if t is None:
            return None
        if isinstance(t, time):
            return t.hour * 60 + t.minute
        if isinstance(t, str):
            try:
                parts = t.split(":")
                if len(parts) >= 2:
                    h = int(parts[0])
                    m = int(parts[1])
                    return h * 60 + m
            except Exception:
                return None
        return None

In [64]:
data_class =  ORDataModel(db=db, vehicles=veh, orders=orders)

In [65]:
data = data_class.get_data()

In [66]:
len(data['matrix'])

11

In [67]:
len(data['node_mapping'])

11

In [68]:
len(data['time_windows'])

10

In [69]:
data['time_windows']

{1: (0, 1440),
 6: (0, 1440),
 7: (0, 1440),
 17: (0, 1440),
 18: (0, 1440),
 9: (0, 1440),
 23: (240, 390),
 22: (0, 1440),
 21: (0, 1440),
 20: (0, 1440)}

In [70]:
len(data['penalties'])

11

In [72]:
data['geo_constrains']

[{'fromId': 20,
  'toId': 13,
  'fromCode': '1320',
  'toCode': '1105',
  'restrictedVehicle': 4},
 {'fromId': 6,
  'toId': 22,
  'fromCode': '105',
  'toCode': 'SCA2',
  'restrictedVehicle': 6},
 {'fromId': 22,
  'toId': 4,
  'fromCode': 'SCA2',
  'toCode': '5',
  'restrictedVehicle': 1},
 {'fromId': 7,
  'toId': 11,
  'fromCode': '106',
  'toCode': '1005',
  'restrictedVehicle': None}]

In [ ]:
data['order_groups']

{1: [17, 6, 7], 2: [22, 23]}

In [113]:
date.today()

datetime.date(2025, 11, 2)

In [ ]:
req = {"vehicles":[{"vehicle_id":1}, {"vehicle_id":2, "predefined_route_id": 1}], "selected_order_id":[1,2,3,4,5,6,7,8,9]}

In [1]:
%cd ..

c:\Users\HP\OneDrive - University of Moratuwa\Desktop\TMS-Project\TMS-Backend


In [34]:
from sqlalchemy.orm import Session
from typing import Optional, Union, List, Dict
from datetime import datetime, time
from src.database import models
from src.core.matrix_manager import DistanceMatrixManager


class ORDataModel:
    def __init__(self, db: Session, vehicles: List[models.Vehicles], orders: List[models.Order], use_time_windows: bool=False, depot_id: int=1):
        self.db = db
        self.depot_id = depot_id
        
        # Store orders for mapping
        self.orders = orders
        
        # Keep one node per order (not per unique shop)
        self.shop_ids = [o.shop_id for o in orders]  # Can have duplicates
        self.all_nodes = [self.depot_id] + self.shop_ids  # depot = 1
        
        self.distance_matrix, self.time_matrix = self._get_matrix(all_nodes=self.all_nodes)
        self.use_time = self._use_time_window(orders) or use_time_windows
        self.max_distance_per_vehicle, self.max_visits_per_vehicle = self._get_vehicle_constrains(vehicles=vehicles)
        self.penalties = self._build_penalties(orders=orders)
        self.time_windows = self._build_time_windows(orders=orders, use_tw=self.use_time)

        self.order_groups = self._fetch_order_groups(orders)
        self.geo_constrains = self._fetch_geo_constraints(self.db)
        
        # Build order mapping: node_index -> order info
        self.order_map = self._build_order_map(orders)
        
        self.matrix = self.time_matrix if self.use_time else self.distance_matrix
    
    
    def get_data(self):
        return {
            "matrix": self.matrix,
            "demands":[0] + [1]*len(self.shop_ids),  # Each order = 1 visit
            "node_mapping":self.all_nodes,
            "num_vehicles":len(self.max_distance_per_vehicle),
            "depot": 0,
            "max_distance_per_vehicle": self.max_distance_per_vehicle,
            "max_visits_per_vehicle":self.max_visits_per_vehicle,
            "penalties":self.penalties,
            "order_groups": self.order_groups, 
            "geo_constrains": self.geo_constrains,
            "time_windows":self.time_windows,
            "use_time_matrix":self.use_time,
            "order_map": self.order_map,
            "distance_matrix": self.distance_matrix
        }
    
    def _get_matrix(self, all_nodes: List[int]):       
        matrix_mgr = DistanceMatrixManager(self.db)
        
        distance_matrix = matrix_mgr.get_distance_matrix_as_array(all_nodes)
        time_matrix = matrix_mgr.get_time_matrix_as_array(all_nodes)
        
        return distance_matrix, time_matrix
    
    def _use_time_window(self, orders: List[models.Order]):
        has_time_windows = any(
            o.time_window_start is not None and o.time_window_end is not None 
            for o in orders)
        
        return has_time_windows
    
    def _get_vehicle_constrains(self, vehicles: List[models.Vehicles]):
        max_distance_per_vehicle = []
        max_visits_per_vehicle = []
        
        for v in vehicles:
            if v.constraint:
                max_distance_per_vehicle.append(int(v.constraint.max_distance or 500))
                max_visits_per_vehicle.append(int(v.constraint.max_visits or 15))
            else:
                max_distance_per_vehicle.append(500)
                max_visits_per_vehicle.append(15)
                
        return max_distance_per_vehicle, max_visits_per_vehicle


    def _build_time_windows(self, orders: List[models.Order], use_tw: bool):
        
        if not use_tw:
            return None
        
        tw = {0: (self.depot_id,( 0, 1440))}
        
        # Each order gets its own time window (even if same shop)
        for o in orders:
            start = self._to_minutes(getattr(o, "time_window_start", None))
            end = self._to_minutes(getattr(o, "time_window_end", None))
            
            if start is not None and end is not None:
                tw[o.id] = (o.shop_id, (start, end))
            else:
                tw[o.id] = (o.shop_id, (0, 1440))
                
        return tw
    
    
    def _build_penalties(self, orders: List[models.Order]):
        
        penalties = [0]  # Depot has 0 penalty
        
        for order in orders:
            if hasattr(order, 'priority') and order.priority:
                priority = order.priority 
            else:
                priority = models.Priority.MEDIUM
            
            if priority == models.Priority.HIGH:
                penalties.append(100000)
            elif priority == models.Priority.LOW:
                penalties.append(500)
            else:
                penalties.append(5000)
                
        return penalties
    
    
    def _fetch_order_groups(self, orders: List[models.Order]) -> Dict[int, List[int]]:
        """Extract order groups from orders.
        
        Orders in the same group must be assigned to the same vehicle.
        
        Returns:
            Dictionary mapping group_id -> [list of shop_ids in that group]
        """
        order_groups = {}
        
        for order in orders:
            if hasattr(order, 'group') and order.group:
                for group in order.group:
                    group_id = group.id
                    if group_id not in order_groups:
                        order_groups[group_id] = []
                    order_groups[group_id].append(order.shop_id)
            
        # Remove duplicates and return only groups with multiple shops
        filtered_groups = {}
        for group_id, shop_ids in order_groups.items():
            unique_shops = list(set(shop_ids))
            if len(unique_shops) > 1:
                filtered_groups[group_id] = unique_shops
        
        return filtered_groups
    
    def _fetch_geo_constraints(self, db: Session) -> List[Dict]:
        """Fetch geo constraints from database."""
        geo_constraints = db.query(models.GeoConstraint).all()
        
        structured_constraints = []
        for gc in geo_constraints:
            constraint = {
                "fromId": gc.start_shop_id,
                "toId": gc.end_shop_id,
                "fromCode": str(gc.start_shop.shop_code) if gc.start_shop else None,
                "toCode": str(gc.end_shop.shop_code) if gc.end_shop else None,
                "restrictedVehicle": gc.vehicle_id
            }
            if constraint["fromId"] is not None and constraint["toId"] is not None:
                structured_constraints.append(constraint)
        
        return structured_constraints
    
    def _build_order_map(self, orders: List[models.Order]) -> Dict[int, Dict]:
        """Build mapping of node_index to order info.
        
        Since each order is a separate node, we map node index (1-based) to order details.
        
        Returns:
            Dictionary mapping node_index -> order info dict
            Example: {
                1: {"id": 1, "order_id": "ORD-001", "shop_id": 5, "priority": "HIGH"},
                2: {"id": 2, "order_id": "ORD-002", "shop_id": 5, "priority": "MEDIUM"},
                3: {"id": 3, "order_id": "ORD-003", "shop_id": 8, "priority": "LOW"}
            }
        """
        order_map = {}
        
        for idx, order in enumerate(orders, start=1):  # Start from 1 (depot is 0)
            order_info = {
                "id": order.id,
                "order_id": order.order_id,
                "shop_id": order.shop_id,
                "priority": order.priority.value if hasattr(order, 'priority') and order.priority else "MEDIUM",
                "po_value": getattr(order, 'po_value', None),
                "volume": getattr(order, 'volume', None),
                "time_window_start": str(order.time_window_start) if order.time_window_start else None,
                "time_window_end": str(order.time_window_end) if order.time_window_end else None
            }
            order_map[idx] = order_info
        
        return order_map
                
    
    def _to_minutes(self, t: Optional[Union[time, str]]) -> Optional[int]:
        """Convert time to minutes since midnight."""
        if t is None:
            return None
        if isinstance(t, time):
            return t.hour * 60 + t.minute
        if isinstance(t, str):
            try:
                parts = t.split(":")
                if len(parts) >= 2:
                    h = int(parts[0])
                    m = int(parts[1])
                    return h * 60 + m
            except Exception:
                return None
        return None

In [35]:
data = ORDataModel(db=db, vehicles=veh, orders=orders).get_data()

In [36]:
data['order_map']

{1: {'id': 3,
  'order_id': '251104003',
  'shop_id': 7,
  'priority': 'medium',
  'po_value': None,
  'volume': None,
  'time_window_start': None,
  'time_window_end': None},
 2: {'id': 4,
  'order_id': '251104004',
  'shop_id': 12,
  'priority': 'medium',
  'po_value': None,
  'volume': None,
  'time_window_start': None,
  'time_window_end': None},
 3: {'id': 5,
  'order_id': '251104005',
  'shop_id': 13,
  'priority': 'medium',
  'po_value': None,
  'volume': None,
  'time_window_start': None,
  'time_window_end': None},
 4: {'id': 6,
  'order_id': '251104006',
  'shop_id': 8,
  'priority': 'medium',
  'po_value': None,
  'volume': None,
  'time_window_start': '10:00:00',
  'time_window_end': '15:30:00'},
 5: {'id': 7,
  'order_id': '251104007',
  'shop_id': 21,
  'priority': 'medium',
  'po_value': None,
  'volume': None,
  'time_window_start': None,
  'time_window_end': None},
 6: {'id': 8,
  'order_id': '251104008',
  'shop_id': 22,
  'priority': 'medium',
  'po_value': None,
  '

In [37]:
len(data['matrix'])

18

In [38]:
len(data['time_windows'])

18

In [39]:
data['time_windows']

{0: (1, (0, 1440)),
 3: (7, (0, 1440)),
 4: (12, (0, 1440)),
 5: (13, (0, 1440)),
 6: (8, (600, 930)),
 7: (21, (0, 1440)),
 8: (22, (0, 1440)),
 9: (23, (0, 1440)),
 10: (20, (0, 1440)),
 11: (19, (0, 1440)),
 12: (18, (0, 1440)),
 13: (15, (0, 1440)),
 14: (17, (0, 1440)),
 15: (16, (0, 1440)),
 16: (5, (0, 1440)),
 17: (2, (0, 1440)),
 18: (4, (0, 1440)),
 19: (7, (480, 720))}

In [40]:
shop_lis = []
for ord in data['order_map'].values():
    shop_lis.append(ord.get('shop_id'))

shop_lis

[7, 12, 13, 8, 21, 22, 23, 20, 19, 18, 15, 17, 16, 5, 2, 4, 7]

In [41]:
len(shop_lis)

17